In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.modules import *
from src.models.loss import L1_epsilon_lambda
from dataclasses import dataclass
from torch.functional import F
import torch

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

@dataclass
class SDFTransformerConfig:
    dim_context: int = 4
    dim_input: int = 3
    num_outputs: int = 1
    dim_output: int = 1
    delta: float = 0.1
    dim_hidden: int = 128
    num_ctx_seeds: int = 32
    num_x_seeds: int = 32
    num_heads: int = 1

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.epsilon = None
        self.lambdaa = None
        self.proj_x = nn.Linear(config.dim_input, config.dim_hidden)
        self.proj_ctx = nn.Linear(config.dim_context, config.dim_hidden)
        self.pool_ctx = PMA(config.dim_hidden, config.num_heads, config.num_ctx_seeds)
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads)
        self.dec = nn.Sequential(
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads),
            nn.SiLU(),
            PMA(config.dim_hidden, config.num_heads, config.num_outputs),
            nn.SiLU(),
            nn.Linear(config.dim_hidden, config.dim_hidden),
            nn.ReLU(),
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        x = self.proj_x(x)                                  # [B, 1, H]
        x = x.expand(-1, self.config.num_x_seeds, -1)       # [B, X, H]
        y = self.pool_ctx(self.proj_ctx(context))           # [B, Y, H]
        xy = self.cross(x, y)                               # [B, X, H]
        xy = xy + x                                         # [B, X, H]
        xy = self.dec(xy)

        loss = None
        if labels is not None:
            loss = L1_epsilon_lambda(xy, labels, self.epsilon, self.lambdaa, self.config.delta)
        return {'loss': loss, 'logits': xy}

config = SDFTransformerConfig()
model = SDFTransformer(config).to(device)
print(device)

cuda


In [3]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [4]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2025_01_22_no_curriculum'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2025_01_22_no_curriculum-2025-01-23-07-46-53


In [5]:
from transformers import Trainer, TrainingArguments

batch_size = 64
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

curriculum_schedule = [
    {"epochs": 10, "epsilon": 0.0,   "lambda": 0.0,  'learning_rate': 5e-5, 'resolution': 256},
]

In [6]:
from src.visualization.generate_mesh import generate_meshes
from src.data.load_data import get_data_dir

obj_dir = get_data_dir() / 'intermediate'
format_string_base = "{name}-" + current_date + "-curriculum-"

for i, stage in enumerate(curriculum_schedule):
    model.epsilon = stage['epsilon']
    model.lambdaa = stage['lambda']
    trainer.args.num_train_epochs = stage['epochs']
    trainer.args.learning_rate = stage['learning_rate']
    trainer.train()
    format_string = format_string_base + str(i) + ".obj"
    generate_meshes(model, obj_dir, result_dir, format_string, device,
        batch_size, resolution=stage['resolution'], context_size=256)
train_dataset.close()
val_dataset.close()

  0%|          | 0/117190 [00:00<?, ?it/s]

{'loss': 0.0108, 'grad_norm': 0.34355127811431885, 'learning_rate': 4.999573342435362e-05, 'epoch': 0.0}
{'loss': 0.0078, 'grad_norm': 0.9027493000030518, 'learning_rate': 4.999146684870723e-05, 'epoch': 0.0}
{'loss': 0.0064, 'grad_norm': 0.7875605821609497, 'learning_rate': 4.998720027306084e-05, 'epoch': 0.0}
{'loss': 0.0062, 'grad_norm': 0.11942391842603683, 'learning_rate': 4.998293369741446e-05, 'epoch': 0.0}
{'loss': 0.0069, 'grad_norm': 0.4873679578304291, 'learning_rate': 4.997866712176807e-05, 'epoch': 0.0}
{'loss': 0.0076, 'grad_norm': 0.6002193689346313, 'learning_rate': 4.997440054612169e-05, 'epoch': 0.01}
{'loss': 0.0052, 'grad_norm': 0.33612123131752014, 'learning_rate': 4.9970133970475295e-05, 'epoch': 0.01}
{'loss': 0.0075, 'grad_norm': 0.1183251217007637, 'learning_rate': 4.9965867394828916e-05, 'epoch': 0.01}
{'loss': 0.0055, 'grad_norm': 0.030841905623674393, 'learning_rate': 4.9961600819182524e-05, 'epoch': 0.01}
{'loss': 0.0075, 'grad_norm': 0.05888752639293671, '

  0%|          | 0/2344 [00:00<?, ?it/s]

{'eval_loss': 0.003118994180113077, 'eval_runtime': 98.3995, 'eval_samples_per_second': 1524.398, 'eval_steps_per_second': 23.821, 'epoch': 1.0}
{'loss': 0.0032, 'grad_norm': 0.046507928520441055, 'learning_rate': 4.4999573342435364e-05, 'epoch': 1.0}
{'loss': 0.0031, 'grad_norm': 0.27737534046173096, 'learning_rate': 4.499530676678897e-05, 'epoch': 1.0}
{'loss': 0.0031, 'grad_norm': 0.19756516814231873, 'learning_rate': 4.499104019114259e-05, 'epoch': 1.0}
{'loss': 0.0033, 'grad_norm': 0.1946389079093933, 'learning_rate': 4.49867736154962e-05, 'epoch': 1.0}
{'loss': 0.0031, 'grad_norm': 0.1391858011484146, 'learning_rate': 4.498250703984982e-05, 'epoch': 1.0}
{'loss': 0.0029, 'grad_norm': 0.31913432478904724, 'learning_rate': 4.497824046420343e-05, 'epoch': 1.0}
{'loss': 0.0032, 'grad_norm': 0.4175320565700531, 'learning_rate': 4.497397388855705e-05, 'epoch': 1.01}
{'loss': 0.0029, 'grad_norm': 0.681201696395874, 'learning_rate': 4.496970731291066e-05, 'epoch': 1.01}
{'loss': 0.0036, 

  0%|          | 0/2344 [00:00<?, ?it/s]

...

{'eval_loss': 0.001608179067261517, 'eval_runtime': 101.6483, 'eval_samples_per_second': 1475.676, 'eval_steps_per_second': 23.06, 'epoch': 10.0}
{'train_runtime': 8174.036, 'train_samples_per_second': 917.539, 'train_steps_per_second': 14.337, 'train_loss': 0.0022108236364165165, 'epoch': 10.0}


Processing models:   0%|          | 0/3 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/262144 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/262144 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/262144 [00:00<?, ?it/s]

In [7]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)